In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import sqlite3
import sqllite_handler
import copy

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### transfer below block into analysis.py

In [ ]:
conn = sqlite3.connect('tables/texts_69_76.db')
cur = conn.cursor()

res = cur.execute("SELECT TEXT FROM transcript")
fetched = res.fetchall()
free_text_list = list(map(lambda x: x[0], fetched))

topic_model = BERTopic.load("plots/topic_model_69_76")

doc_df = pd.read_csv('tables/doc_69_76.csv')

messy_doc_topic_df = topic_model.get_document_info(free_text_list)

topic_desc_df = messy_doc_topic_df[['Name','Top_n_words']].drop_duplicates(ignore_index=True)
doc_topic_df = pd.DataFrame({'id_to_text':doc_df['id_to_text'],'assigned_topic':messy_doc_topic_df['Name']})

topic_desc_df.to_csv('tables/topic_descp_69_76.csv')
doc_topic_df.to_csv('tables/doc_topic_69_76.csv')

### construction below

In [2]:
# LDA topic modeling

import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
import spacy
import math

pyLDAvis.enable_notebook()
nlp = spacy.load('en_core_web_sm')

In [6]:
doc_df = pd.read_csv('tables/tables_69_76/doc_69_76v30.csv')
free_text_list = doc_df['text'].values
free_text_list = list(map(lambda x: ' ' if not x or (isinstance(x, float) and math.isnan(x)) else x, free_text_list))

In [7]:
processed_free_text_list = list(map(lambda row: [t.lower_ for t in nlp(row) if not t.is_punct and not t.is_stop], free_text_list))

dictionary = Dictionary(processed_free_text_list)

dictionary.filter_extremes(keep_n=10000)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in processed_free_text_list]

lda = LdaModel(corpus = doc_term_matrix, id2word=dictionary, num_topics=10)

gensimvis.prepare(lda, doc_term_matrix, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.014009 -0.011560       1        1  20.181976
9      0.008188  0.012510       2        1  13.792053
1     -0.008702  0.003447       3        1  12.520402
8     -0.001183 -0.006941       4        1  11.395890
2      0.019019 -0.009487       5        1   9.729934
4     -0.020579  0.004391       6        1   9.378122
5      0.008183  0.014582       7        1   9.338112
6      0.011842 -0.008383       8        1   6.679372
0     -0.003693 -0.001471       9        1   4.356281
3      0.000934  0.002911      10        1   2.627859, topic_info=              Term        Freq       Total Category  logprob  loglift
1531     kissinger  885.000000  885.000000  Default  30.0000  30.0000
3018             k  271.000000  271.000000  Default  29.0000  29.0000
32            nato  640.000000  640.000000  Default  28.0000  28.0000
31             mr.  788.000000  788.000000  Default  27.0000  27.0000
29        minister  457.000000  457.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
1136          said    7.515656  440.369876  Topic10  -5.8788  -0.4316
913          right    6.595941  257.394703  Topic10  -6.0093  -0.0252
84      ambassador    6.711833  292.006968  Topic10  -5.9919  -0.1339
2620       bitsios    6.799866  386.736579  Topic10  -5.9789  -0.4018
1719  negotiations    6.649047  408.721281  Topic10  -6.0013  -0.4796

[785 rows x 6 columns], token_table=      Topic      Freq Term
term                      
60        1  0.178258   12
60        2  0.127327   12
60        3  0.101862   12
60        4  0.152793   12
60        5  0.076396   12
...     ...       ...  ...
1104      6  0.107278  yes
1104      7  0.059599  yes
1104      8  0.055626  yes
1104      9  0.051653  yes
1104     10  0.019866  yes

[3987 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 2, 9, 3, 5, 6, 7, 1, 4])

In [1]:
# extracting particular volumes
import re
import glob
import math
import json
import pandas as pd
import numpy as np
from datetime import datetime
import xml.etree.ElementTree as ET

ns = {'xml': 'http://www.w3.org/XML/1998/namespace',
      'dflt': 'http://www.tei-c.org/ns/1.0',
      'frus':'http://history.state.gov/frus/ns/1.0',
      'xi':'http://www.w3.org/2001/XInclude'
      }

In [14]:
start_year, end_year = 1952, 1988

for file in glob.glob('volumes/frus*'):
    file_start_year = int(file[12:16])
    
    if file_start_year >= start_year and file_start_year<=end_year:

        volume = file[8:-4]

        tree = ET.parse(file)
        root = tree.getroot()